# Concept 1: Long-Term Agent Memory Management

**Objective**: Build a finance assistant that persists client profiles in PostgreSQL and applies intelligent memory policies to maintain relevant context over extended sessions.

**Key Features**:
- 🧠 **Persistent Memory**: Store client facts in PostgreSQL with weights and TTL
- 🔄 **Smart Updates**: Upsert new facts and resolve conflicts intelligently
- ✂️ **Memory Pruning**: Remove stale, expired, or low-value memories
- 📊 **Dynamic Reweighting**: Boost recent, frequent, or pinned memories
- 📋 **Compact Digests**: Maintain focused profile summaries (≤12 items)

**Time**: ~15-20 minutes

**Scenario**: A finance assistant that remembers client goals, risk tolerance, constraints, and bills across multiple sessions.

## 🏗️ Memory Architecture

Each memory entry contains:
- **Core Data**: id, topic, fact_text, source
- **Temporal**: created_at, updated_at, ttl_days
- **Weighting**: weight, pinned, frequency_count
- **Organization**: tags for categorization

## 🎯 Learning Objectives

By the end of this exercise, you will:
1. Design persistent memory schemas for AI agents
2. Implement intelligent memory update and conflict resolution
3. Create memory pruning policies with TTL and weight thresholds
4. Build dynamic reweighting based on recency, frequency, and importance
5. Generate compact memory digests for efficient context usage

In [ ]:
# Import required libraries
import os
import json
import uuid
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, asdict
from enum import Enum

# Database and ORM
import sqlalchemy as sa
from sqlalchemy import create_engine, Column, String, Float, Integer, Boolean, DateTime, Text, JSON
from sqlalchemy.orm import declarative_base, sessionmaker, Session

# OpenAI for LLM
import openai
from openai import OpenAI

# Environment variables
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("🔧 Environment Setup:")
print(f"   ✅ OpenAI API Key: {'✓ Configured' if os.getenv('OPENAI_API_KEY') else '❌ Missing'}")
print("   🗄️ Database: PostgreSQL with SQLAlchemy ORM")
print("   🧠 Memory System: Persistent client profile management")
print("   💰 Domain: Personal finance assistant")

## 🗄️ Memory Database Schema

Let's define our PostgreSQL schema for storing agent memories with all the required fields for intelligent memory management.

In [ ]:
# Database setup
Base = declarative_base()

class MemoryEntry(Base):
    """Database model for agent memory entries"""
    __tablename__ = 'memory_entries'
    
    # YOUR CODE HERE: Define the complete database schema
    # Hint: Include all fields mentioned in the requirements:
    # - id (String(36) primary key for SQLite compatibility)
    # - topic, fact_text, source (core memory data)
    # - created_at, updated_at, ttl_days (temporal metadata)
    # - weight, pinned, frequency_count (weighting and prioritization)
    # - tags (JSON field for organization)
    
    def __repr__(self):
        return f"<MemoryEntry(topic='{self.topic}', weight={self.weight})>"
    
    def is_expired(self) -> bool:
        """Check if memory has expired based on TTL"""
        # YOUR CODE HERE: Implement TTL expiration logic
        # Hint: Compare current time with created_at + ttl_days
        pass
    
    def days_since_creation(self) -> int:
        """Calculate days since memory was created"""
        # YOUR CODE HERE: Calculate days since creation
        pass
    
    def days_since_update(self) -> int:
        """Calculate days since memory was last updated"""
        # YOUR CODE HERE: Calculate days since last update
        pass

@dataclass
class MemoryPolicy:
    """Configuration for memory management policies"""
    # YOUR CODE HERE: Define the memory policy configuration
    # Hint: Include TTL policies, weighting coefficients, and digest settings
    # Look at the requirements for the complete structure:
    # - ttl_policies: Dict mapping topics to TTL days
    # - base_weight, recency_alpha, frequency_beta, pinned_gamma
    # - max_digest_size, min_weight_threshold
    pass

# Create database engine (SQLite for demo, but designed for PostgreSQL)
DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///finance_memory.db")
engine = create_engine(DATABASE_URL, echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create tables
Base.metadata.create_all(bind=engine)

print("🗄️ Database Schema Created:")
print("   📊 MemoryEntry table with full metadata")
print("   ⚙️ Memory policies configured")
print("   🔗 SQLAlchemy ORM ready for PostgreSQL")

## 🧠 Intelligent Memory Manager

Now let's build the core memory management system that can update, prune, and reweight memories intelligently.

In [ ]:
class FinanceMemoryManager:
    """Intelligent memory manager for finance assistant"""
    
    def __init__(self, policy: MemoryPolicy = None):
        self.policy = policy or MemoryPolicy()
        self.session = SessionLocal()
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()
    
    def canonicalize_fact(self, fact_text: str, topic: str) -> str:
        """Use LLM to canonicalize and normalize fact text"""
        # YOUR CODE HERE: Implement LLM-powered fact canonicalization
        # Hint: Create a prompt that asks the LLM to normalize the fact text
        # Use consistent formatting, terminology, and structure
        pass
    
    def detect_conflicts(self, new_fact: str, topic: str) -> List[MemoryEntry]:
        """Detect existing memories that conflict with new fact"""
        # YOUR CODE HERE: Implement conflict detection
        # Hint: 
        # 1. Get all existing memories for the topic
        # 2. Use LLM to analyze if new fact conflicts with existing ones
        # 3. Return list of conflicting memory entries
        pass
    
    def upsert_memory(self, fact_text: str, topic: str, source: str = "user", 
                     tags: List[str] = None, pinned: bool = False) -> Tuple[MemoryEntry, List[str]]:
        """Update or insert memory with conflict resolution"""
        # YOUR CODE HERE: Implement memory upsert logic
        # Hint:
        # 1. Canonicalize the fact text
        # 2. Detect and resolve conflicts
        # 3. Check for duplicates
        # 4. Either update existing or create new memory
        # 5. Return the memory entry and list of actions taken
        pass
    
    def calculate_weight(self, memory: MemoryEntry) -> float:
        """Calculate dynamic weight based on recency, frequency, and importance"""
        # YOUR CODE HERE: Implement weight calculation formula
        # Hint: w = base + α*recency + β*frequency + γ*pinned
        # Use the policy coefficients for weighting
        pass
    
    def reweight_all_memories(self) -> List[str]:
        """Recalculate weights for all memories"""
        # YOUR CODE HERE: Implement memory reweighting
        # Hint:
        # 1. Get all memories from database
        # 2. Calculate new weights for each
        # 3. Update if weight changed significantly
        # 4. Return list of reweighting actions
        pass
    
    def prune_memories(self) -> List[str]:
        """Remove expired and low-value memories"""
        # YOUR CODE HERE: Implement memory pruning
        # Hint:
        # 1. Remove expired memories (TTL-based)
        # 2. Remove low-weight memories if over digest size limit
        # 3. Respect pinned memories
        # 4. Return list of pruning actions
        pass
    
    def generate_profile_digest(self) -> Dict[str, Any]:
        """Generate compact profile digest for LLM context"""
        # YOUR CODE HERE: Implement digest generation
        # Hint:
        # 1. Get top-weighted memories (up to max_digest_size)
        # 2. Group by topic for organization
        # 3. Create bullet-point summary
        # 4. Return structured digest with metadata
        pass
    
    def get_memory_stats(self) -> Dict[str, Any]:
        """Get memory system statistics"""
        # YOUR CODE HERE: Implement statistics collection
        # Hint: Count total memories, by topic, pinned count, average weight
        pass

print("🧠 Memory Manager Template Created:")
print("   ⚠️ Complete the YOUR CODE HERE sections to implement:")
print("   📝 Fact canonicalization with LLM")
print("   🔍 Conflict detection and resolution")
print("   ⚖️ Dynamic weight calculation")
print("   ✂️ Smart memory pruning")
print("   📋 Profile digest generation")

## 💰 Finance Assistant with Memory

Let's create the finance assistant that uses our memory system to provide consistent, context-aware advice.

In [ ]:
class FinanceAssistant:
    """Finance assistant with persistent memory"""
    
    def __init__(self, memory_manager: FinanceMemoryManager):
        self.memory = memory_manager
        self.session_history = []
    
    def process_user_input(self, user_input: str, confirm_changes: bool = True) -> Dict[str, Any]:
        """Process user input and update memory accordingly"""
        # YOUR CODE HERE: Implement user input processing
        # Hint:
        # 1. Extract facts from user input using LLM
        # 2. Process each fact with memory upsert
        # 3. Reweight and prune memories
        # 4. Return results with actions taken
        pass
    
    def _extract_facts(self, user_input: str) -> List[Dict[str, Any]]:
        """Extract structured facts from user input using LLM"""
        # YOUR CODE HERE: Implement fact extraction
        # Hint:
        # 1. Create LLM prompt to extract facts
        # 2. Categorize into topics (goals, risk, constraints, bills, etc.)
        # 3. Return structured JSON with fact details
        pass
    
    def answer_question(self, question: str) -> Dict[str, Any]:
        """Answer financial question using memory context"""
        # YOUR CODE HERE: Implement context-aware question answering
        # Hint:
        # 1. Get current memory digest
        # 2. Use digest as context in LLM prompt
        # 3. Generate personalized financial advice
        # 4. Record the interaction in session history
        pass
    
    def show_memory_summary(self) -> None:
        """Display current memory state"""
        # YOUR CODE HERE: Implement memory summary display
        # Hint: Show digest bullets and memory statistics
        pass

print("💰 Finance Assistant Template Created:")
print("   ⚠️ Complete the YOUR CODE HERE sections to implement:")
print("   📝 User input processing and fact extraction")
print("   🤖 Context-aware question answering")
print("   📊 Memory summary display")

## 🎬 Testing Your Implementation

Once you've completed the YOUR CODE HERE sections, test your implementation with these scenarios:

In [ ]:
# Test your implementation here
def test_memory_system():
    """Test the complete memory system implementation"""
    
    print("🧪 Testing Memory System Implementation")
    print("=" * 50)
    
    # Initialize the system
    # YOUR CODE HERE: Create memory manager and assistant
    
    # Test 1: Client Onboarding
    print("\n📝 Test 1: Client Onboarding")
    onboarding_input = """
    Goal: buy a house in 18 months; target $40k down payment; current savings $12k;
    risk moderate; no crypto; recurring $150 student loan on the 1st.
    """
    
    # YOUR CODE HERE: Process onboarding input and show results
    
    # Test 2: Memory Update
    print("\n🔄 Test 2: Memory Update")
    update_input = "Student loan paid off; remove that bill."
    
    # YOUR CODE HERE: Process update and show changes
    
    # Test 3: Financial Question
    print("\n❓ Test 3: Financial Question")
    question = "I have $8k spare—should I do a 12-month CD or keep in HYSA?"
    
    # YOUR CODE HERE: Answer question using stored context
    
    # Test 4: New Constraint
    print("\n🛡️ Test 4: New Constraint")
    constraint = "New constraint: need 3 months' expenses in cash before locking funds."
    
    # YOUR CODE HERE: Add constraint and re-answer question
    
    print("\n✅ All tests completed!")

# Uncomment to test your implementation
# test_memory_system()

## 🎯 Implementation Checklist

### ✅ Required Components

#### Database Schema (MemoryEntry)
- [ ] Complete SQLAlchemy model with all required fields
- [ ] TTL expiration logic implementation
- [ ] Days calculation methods

#### Memory Policy Configuration
- [ ] TTL policies by topic
- [ ] Weighting formula coefficients
- [ ] Digest size and threshold settings

#### Memory Manager Core Methods
- [ ] LLM-powered fact canonicalization
- [ ] Conflict detection using LLM analysis
- [ ] Memory upsert with conflict resolution
- [ ] Dynamic weight calculation formula
- [ ] Memory reweighting system
- [ ] TTL and weight-based pruning
- [ ] Profile digest generation
- [ ] Memory statistics collection

#### Finance Assistant Features
- [ ] User input processing and fact extraction
- [ ] Context-aware question answering
- [ ] Memory summary display
- [ ] Session history tracking

### 🧪 Success Criteria

Your implementation should demonstrate:
- ✅ **Session A**: Update → digest refresh during onboarding
- ✅ **Session B**: Consistent context usage without re-asking
- ✅ **Memory Operations**: At least one prune and one reweight event
- ✅ **Digest Management**: Capped to ≤12 items with highest weights
- ✅ **Technology Stack**: PostgreSQL + SQLAlchemy + OpenAI integration

### 💡 Implementation Tips

1. **Start Simple**: Begin with basic CRUD operations before adding intelligence
2. **Test Incrementally**: Test each method as you implement it
3. **Use LLM Effectively**: Design clear prompts for fact extraction and conflict detection
4. **Handle Edge Cases**: Account for empty results, API errors, and invalid data
5. **Focus on UX**: Provide clear feedback about memory operations to users

Good luck implementing your intelligent memory system! 🚀